In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
import datetime
from dateutil.relativedelta import relativedelta
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR



In [35]:
data =  pd.read_csv('GOTO.JK.csv')
data = data['Volume']
data

0      3.624955e+09
1      3.994393e+09
2      2.749645e+09
3      4.244450e+09
4      2.290207e+09
           ...     
243    7.172535e+09
244    3.813070e+09
245    4.021625e+09
246             NaN
247    6.155358e+08
Name: Volume, Length: 248, dtype: float64

In [36]:
data.isnull().values.any()

True

In [37]:
data1 = data.dropna()

In [38]:
data1.isnull().values.any()

False

In [39]:
# data_training,data_test = train_test_split(data1, train_size=0.8, test_size=0.2, shuffle=False, random_state=42)


In [40]:
n = len(data1)
sizeTrain = (round(n*0.8))
data_training = pd.DataFrame(data1[:sizeTrain])
data_test = pd.DataFrame(data1[sizeTrain:])

In [41]:
data_training_new = pd.DataFrame(data_training)
data_test_new = pd.DataFrame(data_test)

In [42]:
# Cross Validation
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

In [43]:
print(pd.date_range(start = '2022-06-15',end='2023-06-15', freq='M'))


DatetimeIndex(['2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30',
               '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-31',
               '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31'],
              dtype='datetime64[ns]', freq='M')


In [44]:
# PROSES MEMASUKKAN DATA PREDIKSI KE CSV
# membuat dataframe kosong dengan 3 kolom
data_final = pd.DataFrame(columns=['Month','KNN' , 'RF' , 'SR'])
# berawal pada 1995-september
start_date = datetime.date(2023, 9, 1)
 
# berakhir pada 2023-januari 
end_date = datetime.date(2029, 12, 1)

# loop setiap bulan
delta = relativedelta(months=1)
while start_date <= end_date:
  tanggal = start_date.strftime("%Y-%m")
  # Preprosssing data
  sc = MinMaxScaler()
  train_scaled = sc.fit_transform(data_training_new)
  test_scaled = sc.transform(data_test_new)

  # Proses Pemanggilan Cross Validation 
  X_train, y_train = split_sequence(train_scaled, 6) 
  X_test, y_test = split_sequence(test_scaled, 6) 
  # membuat kolom otomatis
  colom = []
  for c in range(6):
      if(c==0):
          colom.insert(0,"Xt")
      else:
          colom.insert(0,f"Xt-{c}")



  # # Masukkan ke dataframe hasil dari split  squens
  # UNTUK MEMISAHKAN X TRAIN, YTRAIN dan TEST
  newX_train = pd.DataFrame(np.reshape(X_train, (len(X_train) , 6)),columns=colom)
  newY_train = pd.DataFrame(y_train, columns=['Output'])
  new_data_train = pd.concat([newX_train,newY_train],axis=1)
  newX_test = pd.DataFrame(np.reshape(X_test, (len(X_test) , 6)),columns=colom)
  newY_test = pd.DataFrame(y_test, columns=['Output'])
  new_data_test = pd.concat([newX_test,newY_test],axis=1)

  # EKSEKUSI KE model
  # predict knn
  knn = KNeighborsRegressor(n_neighbors = 4)
  knn.fit(newX_train,newY_train)

  # Proses Prediksi
  y_pred = knn.predict(newX_test)
  y_pred_shape = y_pred.reshape(-1,1)
  y_pred_inverse_knn = sc.inverse_transform(y_pred_shape)
  

  # Predict SVR dengan Bagging Regressor
  estimators = [('lr', RidgeCV()),
                ('svr', LinearSVR(random_state=42))]
  reg = StackingRegressor(
            estimators,
            final_estimator=RandomForestRegressor(n_estimators=10,
                                                random_state=42)).fit(newX_train, newY_train.values.ravel())

  # Proses Prediksi
  y_pred = regr.predict(newX_test)
  y_pred_shape = y_pred.reshape(-1,1)
  y_pred_inverse_svr = sc.inverse_transform(y_pred_shape)
  
  

  # Predict dengan from RandomForestRegressor
  regr = RandomForestRegressor(max_depth=8, random_state=0).fit(newX_train, newY_train.values.ravel())

  # Proses Prediksi
  y_pred = regr.predict(newX_test)
  y_pred_shape = y_pred.reshape(-1,1)
  y_pred_inverse_rf = sc.inverse_transform(y_pred_shape)
  
   ## Proses Memasukkan Y_pred terakhir ke dataframe
  data_final = pd.concat([data_final,pd.DataFrame({'Month':tanggal,'KNN':np.round(y_pred_inverse_knn[-1]),'SVR':np.round(y_pred_inverse_svr[-1]),'RF':np.round(y_pred_inverse_rf[-1])})],axis=0)
  
  # mean dari 3 model dimasukkan ke dataframe data_test_new 
  data_test_new = pd.concat([data_test_new,pd.DataFrame({'Volume':np.round(np.mean([y_pred_inverse_knn[-1],y_pred_inverse_svr[-1],y_pred_inverse_rf[-1]]))},index=  [0] )],axis=0)
  start_date += delta

# save data to csv
data_final.to_csv('prediksiluck.csv', index=False)
data_test_new.to_csv('datatestluck.csv', index=False)

data_test_new

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

,Volume
198,3.269322e+09
199,1.562022e+09
200,1.526839e+09
201,2.192203e+09
202,2.525410e+09
...,...
0,2.931251e+09
0,3.160227e+09
0,3.803592e+09
0,1.505528e+10
